#Packages

In [51]:
install.packages("googlesheets4")
install.packages("dplyr")
install.packages("tidyverse")
install.packages("MASS")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [52]:
library(googlesheets4)
library(dplyr)
library(tidyverse)
library(MASS)

#Data

In [63]:
gs4_auth()
sheet_url <- 'https://docs.google.com/spreadsheets/d/1-esORqcQagHMnPjT1VlHSjNeAMHNG0bIi4EX1n6EuNU/edit'

The googlesheets4 package is requesting access to your Google account.
Enter '1' to start a new auth process or select a pre-authorized account.
1: Send me to the browser for a new auth process.
2: z.broszczak@student.uw.edu.pl


Selection: 2


In [64]:
sheet1 <- read_sheet(sheet_url, sheet = 1)
sheet2 <- read_sheet(sheet_url, sheet = 2)
sheet3 <- read_sheet(sheet_url, sheet = 3)
sheet4 <- read_sheet(sheet_url, sheet = 4)

✔ Reading from "Badanie Licencjackie Zofia Broszczak (Odpowiedzi)".

✔ Range ''Online bezpośrednie''.

✔ Reading from "Badanie Licencjackie Zofia Broszczak (Odpowiedzi)".

✔ Range ''Offline bezpośrednie''.

✔ Reading from "Badanie Licencjackie Zofia Broszczak (Odpowiedzi)".

✔ Range ''Offline RR''.

✔ Reading from "Badanie Licencjackie Zofia Broszczak (Odpowiedzi)".

✔ Range ''Online RR''.



In [65]:
dane <- bind_rows(sheet1, sheet2, sheet3, sheet4)

In [66]:
freq <- table(dane[c("Odp", "Grupa")])
freq2 <- table(dane[c("Grupa")])

In [67]:
freq

     Grupa
Odp    1  2  3  4
  Nie 12  8  8  9
  Tak  8  5  6 14

In [68]:
freq2

Grupa
 1  2  3  4 
20 13 14 23 

In [69]:
freq_df <- as.data.frame.matrix(freq)
total_counts <- as.numeric(freq2)
normalized_freq_df <- sweep(freq_df, 2, total_counts, "/")
normalized_freq <- as.table(as.matrix(normalized_freq_df))

print(normalized_freq)

            1         2         3         4
Nie 0.6000000 0.6153846 0.5714286 0.3913043
Tak 0.4000000 0.3846154 0.4285714 0.6086957


#Example

In [70]:
total_counts <- c(20, 13, 14, 23)

subtract_1_6_group3 <- (total_counts[3])/6
subtract_1_6_group4 <- (total_counts[4])/6

subtract_1_6_group3
subtract_1_6_group4

[1] 2.333333

[1] 3.833333

In [71]:
freq["Tak", "3"] <- freq["Tak", "3"] - subtract_1_6_group3
freq["Nie", "3"] <- freq["Nie", "3"] - subtract_1_6_group3

freq["Tak", "4"] <- freq["Tak", "4"] - subtract_1_6_group4
freq["Nie", "4"] <- freq["Nie", "4"] - subtract_1_6_group4
print(freq)


     Grupa
Odp           1         2         3         4
  Nie 12.000000  8.000000  5.666667  5.166667
  Tak  8.000000  5.000000  3.666667 10.166667


In [72]:
freq2["3"] <- freq[1,3] +freq[2,3]
freq2["4"] <- freq[1,4] +freq[2,4]


In [73]:
freq_df <- as.data.frame.matrix(freq)
total_counts <- as.numeric(freq2)
normalized_freq_df <- sweep(freq_df, 2, total_counts, "/")
normalized_freq <- as.table(as.matrix(normalized_freq_df))

print(normalized_freq)

            1         2         3         4
Nie 0.6000000 0.6153846 0.6071429 0.3369565
Tak 0.4000000 0.3846154 0.3928571 0.6630435


In [74]:
freq2

        1         2         3         4 
20.000000 13.000000  9.333333 15.333333 

#H1 : Online directly vs Offline directly (GROUP 1 VS GROUP 2).

In [75]:
p12 = (freq[2,2] + freq[2,1]) / (freq2[1] + freq2[2])
print(p12)

        1 
0.3939394 


In [76]:
observed_diff_p12 = abs(freq[2,1]/freq2[1]-freq[2,2]/freq2[2])
observed_diff_p12 = format(observed_diff_p12, scientific = FALSE)
print(observed_diff_p12)

           1 
"0.01538462" 


In [77]:
l1<- 0:20

lista_gr1_p12 <- numeric(length(l1))

for (i in l1) {
  lista_gr1_p12[i + 1] <- dbinom(i, size = 20, prob = p12)
}

print(lista_gr1_p12)

 [1] 4.470140e-05 5.811182e-04 3.588405e-03 1.399478e-02 3.866058e-02
 [6] 8.041400e-02 1.306727e-01 1.698746e-01 1.794300e-01 1.555060e-01
[11] 1.111868e-01 6.570129e-02 3.202938e-02 1.281175e-02 4.163819e-03
[16] 1.082593e-03 2.199017e-04 3.363203e-05 3.643469e-06 2.492900e-07
[21] 8.101926e-09


In [78]:
l2 <- 0:13

lista_gr2_p12 <- numeric(length(l2))

for (i in l2) {
  lista_gr2_p12[i + 1] <- dbinom(i, size = 13, prob = p12)
}

print(lista_gr2_p12)

 [1] 1.488363e-03 1.257666e-02 4.904899e-02 1.169001e-01 1.899626e-01
 [6] 2.222563e-01 1.926221e-01 1.252044e-01 6.103713e-02 2.204119e-02
[11] 5.730709e-03 1.015898e-03 1.100557e-04 5.502783e-06


In [79]:
result_matrix_p12 <- outer(lista_gr1_p12, lista_gr2_p12, FUN = "*")
result_matrix_p12 <- round(result_matrix_p12, digits = 6)
result_matrix_p12 <- format(result_matrix_p12, scientific = FALSE)

print(result_matrix_p12)

      [,1]       [,2]       [,3]       [,4]       [,5]       [,6]      
 [1,] "0.000000" "0.000001" "0.000002" "0.000005" "0.000008" "0.000010"
 [2,] "0.000001" "0.000007" "0.000029" "0.000068" "0.000110" "0.000129"
 [3,] "0.000005" "0.000045" "0.000176" "0.000419" "0.000682" "0.000798"
 [4,] "0.000021" "0.000176" "0.000686" "0.001636" "0.002658" "0.003110"
 [5,] "0.000058" "0.000486" "0.001896" "0.004519" "0.007344" "0.008593"
 [6,] "0.000120" "0.001011" "0.003944" "0.009400" "0.015276" "0.017873"
 [7,] "0.000194" "0.001643" "0.006409" "0.015276" "0.024823" "0.029043"
 [8,] "0.000253" "0.002136" "0.008332" "0.019858" "0.032270" "0.037756"
 [9,] "0.000267" "0.002257" "0.008801" "0.020975" "0.034085" "0.039879"
[10,] "0.000231" "0.001956" "0.007627" "0.018179" "0.029540" "0.034562"
[11,] "0.000165" "0.001398" "0.005454" "0.012998" "0.021121" "0.024712"
[12,] "0.000098" "0.000826" "0.003223" "0.007680" "0.012481" "0.014603"
[13,] "0.000048" "0.000403" "0.001571" "0.003744" "0.006084" "0.

In [80]:
write.matrix(result_matrix_p12, file="M_p12.csv")

Create the mask and apply the mask


In [81]:
n_rows_p12 <- nrow(result_matrix_p12)
n_cols_p12 <- ncol(result_matrix_p12)
print(n_rows_p12)

mask_p12 <- outer(1:n_rows_p12, 1:n_cols_p12, FUN = function(i, j) (((i-1) / 20) - ((j-1) / 13)) > observed_diff_p12)
print(mask_p12)

final_values_p12 = result_matrix_p12[mask_p12]
print(final_values_p12)

[1] 21
       [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9] [,10] [,11] [,12]
 [1,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [2,]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [3,]  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [4,]  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [5,]  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [6,]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [7,]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [8,]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [9,]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[10,]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE
[11,]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE
[12,]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FAL

In [83]:
sum(as.numeric(final_values_p12))

[1] 0.447811

p.value = 0.45 - cannot reject the null hypothesis - no significant differences between groups 1 and 2

#H2 : Offline with Randomized Response vs Ofline directly (GROUP 3 VS GROUP 2).

In [84]:
p23 = (freq[2,2] + freq[2,3]) / (freq2[2] + freq2[3])
print(p23)
observed_diff_p23 = abs(freq[2,2]/freq2[2]-freq[2,3]/freq2[3])
print(observed_diff_p23)

        2 
0.3880597 
          2 
0.008241758 


In [85]:
p3 = 1/6 +p23*4/6
print(p3)

        2 
0.4253731 


In [86]:
l2 <- 0:13

lista_gr2_p23 <- numeric(length(l2))

for (i in l2) {
  lista_gr2_p23[i + 1] <- dbinom(i, size = 13, prob = p23)
}

print(lista_gr2_p23)

 [1] 1.687399e-03 1.391075e-02 5.292871e-02 1.230700e-01 1.951110e-01
 [6] 2.227120e-01 1.883094e-01 1.194157e-01 5.679527e-02 2.000917e-02
[11] 5.075498e-03 8.778023e-04 9.277585e-05 4.525651e-06


In [87]:
l3<- 0:14

lista_gr3_p23 <- numeric(length(l3))

for (i in l3) {
  lista_gr3_p23[i + 1] <- dbinom(i, size = 14, prob = p3)
}

print(lista_gr3_p23)

 [1] 4.279598e-04 4.435220e-03 2.134089e-02 6.319122e-02 1.286393e-01
 [6] 1.904529e-01 2.114770e-01 1.789119e-01 1.158861e-01 5.719054e-02
[11] 2.116793e-02 5.698096e-03 1.054518e-03 1.200949e-04 6.350103e-06


In [92]:
result_matrix_p23 <- outer(lista_gr2_p23, lista_gr3_p23, FUN = "*")
result_matrix_p23 <- round(result_matrix_p23, digits = 6)
result_matrix_p23 <- format(result_matrix_p23, scientific = FALSE)
result_matrix_p23 <- as.numeric(result_matrix_p23)
print(result_matrix_p23)

  [1] 0.000001 0.000006 0.000023 0.000053 0.000083 0.000095 0.000081 0.000051
  [9] 0.000024 0.000009 0.000002 0.000000 0.000000 0.000000 0.000007 0.000062
 [17] 0.000235 0.000546 0.000865 0.000988 0.000835 0.000530 0.000252 0.000089
 [25] 0.000023 0.000004 0.000000 0.000000 0.000036 0.000297 0.001130 0.002626
 [33] 0.004164 0.004753 0.004019 0.002548 0.001212 0.000427 0.000108 0.000019
 [41] 0.000002 0.000000 0.000107 0.000879 0.003345 0.007777 0.012329 0.014073
 [49] 0.011899 0.007546 0.003589 0.001264 0.000321 0.000055 0.000006 0.000000
 [57] 0.000217 0.001789 0.006809 0.015832 0.025099 0.028650 0.024224 0.015362
 [65] 0.007306 0.002574 0.000653 0.000113 0.000012 0.000001 0.000321 0.002649
 [73] 0.010080 0.023439 0.037159 0.042416 0.035864 0.022743 0.010817 0.003811
 [81] 0.000967 0.000167 0.000018 0.000001 0.000357 0.002942 0.011193 0.026026
 [89] 0.041261 0.047098 0.039823 0.025254 0.012011 0.004231 0.001073 0.000186
 [97] 0.000020 0.000001 0.000302 0.002489 0.009470 0.022019 0.03

In [93]:
result_matrix_p23 <- round(result_matrix_p23, digits = 6)
result_matrix_p23 <- format(result_matrix_p23, scientific = FALSE)
print(result_matrix_p23)
write.matrix(result_matrix_p23, file="M_p23.csv")

  [1] "0.000001" "0.000006" "0.000023" "0.000053" "0.000083" "0.000095"
  [7] "0.000081" "0.000051" "0.000024" "0.000009" "0.000002" "0.000000"
 [13] "0.000000" "0.000000" "0.000007" "0.000062" "0.000235" "0.000546"
 [19] "0.000865" "0.000988" "0.000835" "0.000530" "0.000252" "0.000089"
 [25] "0.000023" "0.000004" "0.000000" "0.000000" "0.000036" "0.000297"
 [31] "0.001130" "0.002626" "0.004164" "0.004753" "0.004019" "0.002548"
 [37] "0.001212" "0.000427" "0.000108" "0.000019" "0.000002" "0.000000"
 [43] "0.000107" "0.000879" "0.003345" "0.007777" "0.012329" "0.014073"
 [49] "0.011899" "0.007546" "0.003589" "0.001264" "0.000321" "0.000055"
 [55] "0.000006" "0.000000" "0.000217" "0.001789" "0.006809" "0.015832"
 [61] "0.025099" "0.028650" "0.024224" "0.015362" "0.007306" "0.002574"
 [67] "0.000653" "0.000113" "0.000012" "0.000001" "0.000321" "0.002649"
 [73] "0.010080" "0.023439" "0.037159" "0.042416" "0.035864" "0.022743"
 [79] "0.010817" "0.003811" "0.000967" "0.000167" "0.000018" "0.

In [94]:
l3total <- c(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14)

x <- subtract_1_6_group3
print(x)

process_value <- function(value, x) {
  if (value < x) {
    return(0)
  } else {
    return(value - x)
  }
}

pest3_list <- vector("numeric", length = 0)

for (i in l3total) {
  j <- 14 - i
  i <- process_value(i, x)
  j <- process_value(j, x)
  p <- ifelse(i + j == 0, 0, i / (i + j))
  pest3_list <- c(pest3_list, p)
}

print(pest3_list)

[1] 2.333333
 [1] 0.00000000 0.00000000 0.00000000 0.07142857 0.17857143 0.28571429
 [7] 0.39285714 0.50000000 0.60714286 0.71428571 0.82142857 0.92857143
[13] 1.00000000 1.00000000 1.00000000


In [95]:
num_vector <- 0:13

pest2_list <- num_vector / 13

print(pest2_list)
print(pest2_list[2])

 [1] 0.00000000 0.07692308 0.15384615 0.23076923 0.30769231 0.38461538
 [7] 0.46153846 0.53846154 0.61538462 0.69230769 0.76923077 0.84615385
[13] 0.92307692 1.00000000
[1] 0.07692308


In [96]:
difference_matrix <- outer(pest2_list, pest3_list, FUN = function(p2, p3) p3 - p2)

print("Difference matrix (p3 - p2):")
print(difference_matrix)

[1] "Difference matrix (p3 - p2):"
             [,1]        [,2]        [,3]         [,4]        [,5]        [,6]
 [1,]  0.00000000  0.00000000  0.00000000  0.071428571  0.17857143  0.28571429
 [2,] -0.07692308 -0.07692308 -0.07692308 -0.005494505  0.10164835  0.20879121
 [3,] -0.15384615 -0.15384615 -0.15384615 -0.082417582  0.02472527  0.13186813
 [4,] -0.23076923 -0.23076923 -0.23076923 -0.159340659 -0.05219780  0.05494505
 [5,] -0.30769231 -0.30769231 -0.30769231 -0.236263736 -0.12912088 -0.02197802
 [6,] -0.38461538 -0.38461538 -0.38461538 -0.313186813 -0.20604396 -0.09890110
 [7,] -0.46153846 -0.46153846 -0.46153846 -0.390109890 -0.28296703 -0.17582418
 [8,] -0.53846154 -0.53846154 -0.53846154 -0.467032967 -0.35989011 -0.25274725
 [9,] -0.61538462 -0.61538462 -0.61538462 -0.543956044 -0.43681319 -0.32967033
[10,] -0.69230769 -0.69230769 -0.69230769 -0.620879121 -0.51373626 -0.40659341
[11,] -0.76923077 -0.76923077 -0.76923077 -0.697802198 -0.59065934 -0.48351648
[12,] -0.84615385

Create the mask matrix based on the condition p3 - p2 > x

In [97]:
mask <- difference_matrix > observed_diff_p23

print("Mask matrix (p3 - p2 > observed_diff_p23):")
print(mask)

[1] "Mask matrix (p3 - p2 > observed_diff_p23):"
       [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9] [,10] [,11] [,12]
 [1,] FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [2,] FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [3,] FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [4,] FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [5,] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [6,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
 [7,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
 [8,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE
 [9,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE
[10,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE
[11,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
[12,] FALSE FAL

Apply the mask to the result matrix and sum the filtered values


In [99]:
filtered_values_23 <- as.numeric(result_matrix_p23[mask])

print("Filtered values_23:")
print(filtered_values_23)

sum(filtered_values_23)

[1] "Filtered values_23:"
  [1] 0.000107 0.000217 0.001789 0.006809 0.000321 0.002649 0.010080 0.023439
  [9] 0.000357 0.002942 0.011193 0.026026 0.041261 0.000302 0.002489 0.009470
 [17] 0.022019 0.034908 0.039846 0.033691 0.000196 0.001612 0.006134 0.014262
 [25] 0.022611 0.025809 0.021822 0.013839 0.000097 0.000796 0.003027 0.007038
 [33] 0.011159 0.012737 0.010770 0.006829 0.003248 0.001144 0.000036 0.000294
 [41] 0.001120 0.002605 0.004130 0.004714 0.003986 0.002528 0.001202 0.000424
 [49] 0.000107 0.000010 0.000079 0.000302 0.000701 0.001112 0.001269 0.001073
 [57] 0.000680 0.000324 0.000114 0.000029 0.000005 0.000002 0.000015 0.000056
 [65] 0.000130 0.000206 0.000235 0.000199 0.000126 0.000060 0.000021 0.000005
 [73] 0.000001 0.000000 0.000000 0.000002 0.000006 0.000015 0.000023 0.000027
 [81] 0.000023 0.000014 0.000007 0.000002 0.000001 0.000000 0.000000 0.000000
 [89] 0.000000 0.000000 0.000001 0.000001 0.000001 0.000001 0.000001 0.000000
 [97] 0.000000 0.000000 0.000000 0.000

[1] 0.46107

p.value = 0.46 -  cannot reject the null hypothesis - no significant differences between groups 2 and 3.

#H3: online bezp vs online RR (GRUPA 1 VS GRUPA 4).

joined p for goup 1 and 4

In [100]:
p14 = (freq[2,1] + freq[2,4]) / (freq2[1] + freq2[4])
print(p14)

        1 
0.5141509 


observed difference in answers between group 1 and 4

In [101]:
observed_diff_p14 = abs(freq[2,1]/freq2[1]-freq[2,4]/freq2[4])
print(observed_diff_p14)

        1 
0.2630435 


In [102]:
p4 = 1/6 +p14*4/6
print(p4)

       1 
0.509434 


In [103]:
l1 <- 0:20

lista_gr1_p14 <- numeric(length(l1))

for (i in l1) {
  lista_gr1_p14[i + 1] <- dbinom(i, size = 20, prob = p14)
}

print(lista_gr1_p14)

 [1] 5.370655e-07 1.136702e-05 1.142772e-04 7.256045e-04 3.263459e-03
 [6] 1.105140e-02 2.923793e-02 6.188223e-02 1.064164e-01 1.501539e-01
[11] 1.747908e-01 1.681571e-01 1.334645e-01 8.691637e-02 4.598973e-02
[16] 1.946750e-02 6.437977e-03 1.603060e-03 2.827404e-04 3.149586e-05
[21] 1.666528e-06


In [104]:
l4 <- 0:23

lista_gr4_p14 <- numeric(length(l4))

for (i in l4) {
  lista_gr4_p14[i + 1] <- dbinom(i, size = 23, prob = p4)
}

print(lista_gr4_p14)

 [1] 7.692048e-08 1.837216e-06 2.098666e-05 1.525569e-04 7.921223e-04
 [6] 3.125836e-03 9.738183e-03 2.455948e-02 5.100816e-02 8.828335e-02
[11] 1.283504e-01 1.575210e-01 1.635794e-01 1.437370e-01 1.066181e-01
[16] 6.643127e-02 3.449316e-02 1.474934e-02 5.105540e-03 1.395239e-03
[21] 2.897803e-04 4.298939e-05 4.058439e-06 1.832405e-07


In [105]:
result_matrix_p14 <- outer(lista_gr1_p14, lista_gr4_p14, FUN = "*")
print(result_matrix_p14)

              [,1]         [,2]         [,3]         [,4]         [,5]
 [1,] 4.131134e-14 9.867055e-13 1.127121e-11 8.193304e-11 4.254216e-10
 [2,] 8.743565e-13 2.088367e-11 2.385558e-10 1.734117e-09 9.004068e-09
 [3,] 8.790254e-12 2.099518e-10 2.398296e-09 1.743377e-08 9.052148e-08
 [4,] 5.581385e-11 1.333092e-09 1.522802e-08 1.106960e-07 5.747675e-07
 [5,] 2.510268e-10 5.995679e-09 6.848911e-08 4.978631e-07 2.585059e-06
 [6,] 8.500792e-10 2.030382e-08 2.319320e-07 1.685968e-06 8.754062e-06
 [7,] 2.248996e-09 5.371640e-08 6.136066e-07 4.460448e-06 2.316002e-05
 [8,] 4.760011e-09 1.136910e-07 1.298701e-06 9.440560e-06 4.901829e-05
 [9,] 8.185601e-09 1.955099e-07 2.233325e-06 1.623456e-05 8.429481e-05
[10,] 1.154991e-08 2.758652e-07 3.151229e-06 2.290701e-05 1.189402e-04
[11,] 1.344499e-08 3.211285e-07 3.668275e-06 2.666554e-05 1.384557e-04
[12,] 1.293472e-08 3.089409e-07 3.529056e-06 2.565352e-05 1.332010e-04
[13,] 1.026615e-08 2.452031e-07 2.800974e-06 2.036092e-05 1.057202e-04
[14,] 

In [106]:
result_matrix_p14 <- round(result_matrix_p14, digits = 6)
result_matrix_p14 <- format(result_matrix_p14, scientific = FALSE)
print(result_matrix_p14)
write.matrix(result_matrix_p14, file="M_p14.csv")

      [,1]       [,2]       [,3]       [,4]       [,5]       [,6]      
 [1,] "0.000000" "0.000000" "0.000000" "0.000000" "0.000000" "0.000000"
 [2,] "0.000000" "0.000000" "0.000000" "0.000000" "0.000000" "0.000000"
 [3,] "0.000000" "0.000000" "0.000000" "0.000000" "0.000000" "0.000000"
 [4,] "0.000000" "0.000000" "0.000000" "0.000000" "0.000001" "0.000002"
 [5,] "0.000000" "0.000000" "0.000000" "0.000000" "0.000003" "0.000010"
 [6,] "0.000000" "0.000000" "0.000000" "0.000002" "0.000009" "0.000035"
 [7,] "0.000000" "0.000000" "0.000001" "0.000004" "0.000023" "0.000091"
 [8,] "0.000000" "0.000000" "0.000001" "0.000009" "0.000049" "0.000193"
 [9,] "0.000000" "0.000000" "0.000002" "0.000016" "0.000084" "0.000333"
[10,] "0.000000" "0.000000" "0.000003" "0.000023" "0.000119" "0.000469"
[11,] "0.000000" "0.000000" "0.000004" "0.000027" "0.000138" "0.000546"
[12,] "0.000000" "0.000000" "0.000004" "0.000026" "0.000133" "0.000526"
[13,] "0.000000" "0.000000" "0.000003" "0.000020" "0.000106" "0.

In [107]:
l4total <- 0:23

x <- subtract_1_6_group4
print(x)

process_value <- function(value, x) {
  if (value < x) {
    return(0)
  } else {
    return(value - x)
  }
}

pest4_list <- vector("numeric", length = 0)

for (i in l4total) {
  j <- 23 - i
  i <- process_value(i, x)
  j <- process_value(j, x)
  p <- ifelse(i + j == 0, 0, i / (i + j))
  pest4_list <- c(pest4_list, p)
}

print(pest4_list)

[1] 3.833333
 [1] 0.00000000 0.00000000 0.00000000 0.00000000 0.01086957 0.07608696
 [7] 0.14130435 0.20652174 0.27173913 0.33695652 0.40217391 0.46739130
[13] 0.53260870 0.59782609 0.66304348 0.72826087 0.79347826 0.85869565
[19] 0.92391304 0.98913043 1.00000000 1.00000000 1.00000000 1.00000000


In [108]:
num_vector_1 <- 0:20

pest1_list <- num_vector_1 / 20

print(pest1_list)

 [1] 0.00 0.05 0.10 0.15 0.20 0.25 0.30 0.35 0.40 0.45 0.50 0.55 0.60 0.65 0.70
[16] 0.75 0.80 0.85 0.90 0.95 1.00


In [109]:
difference_matrix_14 <- outer(pest1_list, pest4_list, FUN = function(p1, p4) p4 - p1)

print("Difference matrix_14 (p4 - p1):")
print(difference_matrix_14)

[1] "Difference matrix_14 (p4 - p1):"
       [,1]  [,2]  [,3]  [,4]        [,5]        [,6]         [,7]         [,8]
 [1,]  0.00  0.00  0.00  0.00  0.01086957  0.07608696  0.141304348  0.206521739
 [2,] -0.05 -0.05 -0.05 -0.05 -0.03913043  0.02608696  0.091304348  0.156521739
 [3,] -0.10 -0.10 -0.10 -0.10 -0.08913043 -0.02391304  0.041304348  0.106521739
 [4,] -0.15 -0.15 -0.15 -0.15 -0.13913043 -0.07391304 -0.008695652  0.056521739
 [5,] -0.20 -0.20 -0.20 -0.20 -0.18913043 -0.12391304 -0.058695652  0.006521739
 [6,] -0.25 -0.25 -0.25 -0.25 -0.23913043 -0.17391304 -0.108695652 -0.043478261
 [7,] -0.30 -0.30 -0.30 -0.30 -0.28913043 -0.22391304 -0.158695652 -0.093478261
 [8,] -0.35 -0.35 -0.35 -0.35 -0.33913043 -0.27391304 -0.208695652 -0.143478261
 [9,] -0.40 -0.40 -0.40 -0.40 -0.38913043 -0.32391304 -0.258695652 -0.193478261
[10,] -0.45 -0.45 -0.45 -0.45 -0.43913043 -0.37391304 -0.308695652 -0.243478261
[11,] -0.50 -0.50 -0.50 -0.50 -0.48913043 -0.42391304 -0.358695652 -0.293478261
[1

Create the mask matrix based on the condition p4 - p1 > obs_diff

In [110]:
mask_14 <- difference_matrix_14 > observed_diff_p14

print("Mask matrix (p4 - p1 > observed_diff_p14):")
print(mask_14)

[1] "Mask matrix (p4 - p1 > observed_diff_p14):"
       [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9] [,10] [,11] [,12]
 [1,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE
 [2,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE
 [3,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
 [4,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
 [5,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
 [6,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [7,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [8,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [9,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[10,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[11,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[12,] FALSE FAL

Apply the mask to the result matrix

In [113]:
filtered_values_14 <- as.numeric(result_matrix_p14[mask_14])

print("Filtered values_14:")
print(filtered_values_14)

[1] "Filtered values_14:"
  [1] 0.000000 0.000000 0.000001 0.000000 0.000001 0.000015 0.000000 0.000002
  [9] 0.000018 0.000114 0.000514 0.000000 0.000002 0.000019 0.000119 0.000534
 [17] 0.001808 0.000000 0.000002 0.000016 0.000104 0.000469 0.001588 0.004203
 [25] 0.000000 0.000001 0.000012 0.000077 0.000348 0.001178 0.003117 0.006598
 [33] 0.000000 0.000001 0.000008 0.000048 0.000217 0.000734 0.001942 0.004111
 [41] 0.007069 0.009975 0.000000 0.000000 0.000004 0.000025 0.000113 0.000381
 [49] 0.001009 0.002135 0.003671 0.005179 0.006029 0.000000 0.000000 0.000002
 [57] 0.000011 0.000048 0.000163 0.000431 0.000913 0.001570 0.002215 0.002578
 [65] 0.002480 0.000000 0.000000 0.000001 0.000004 0.000017 0.000056 0.000149
 [73] 0.000316 0.000543 0.000767 0.000892 0.000859 0.000681 0.000444 0.000000
 [81] 0.000000 0.000000 0.000001 0.000005 0.000015 0.000041 0.000086 0.000148
 [89] 0.000210 0.000244 0.000235 0.000186 0.000121 0.000064 0.000000 0.000000
 [97] 0.000000 0.000000 0.000001 0.000

In [114]:
sum(filtered_values_14)

[1] 0.080335

p.value = 0.08 --> cannot reject the null hypothesis --> there are no statistically significant differences between groups 1 and 4.

#Example for thesis

In [115]:
pA = (1+3)/(4+5)
print(pA)
obs_diff_A = abs(1/4-3/5)
print(obs_diff_A)
lA <- 0:4
print(lA)
lista_A <- numeric(length(lA))

for (i in lA) {
  lista_A[i + 1] <- dbinom(i, size = 4, prob = pA)
}

print(lista_A)

[1] 0.4444444
[1] 0.35
[1] 0 1 2 3 4
[1] 0.09525987 0.30483158 0.36579790 0.19509221 0.03901844


In [116]:
lB <- 0:5

lista_B <- numeric(length(lB))

for (i in lB) {
  lista_B[i + 1] <- dbinom(i, size = 5, prob = pA)
}

print(lista_B)

[1] 0.05292215 0.21168860 0.33870176 0.27096140 0.10838456 0.01734153


In [117]:
m<- outer(lista_A, lista_B, FUN = "*")
print(m)

            [,1]        [,2]       [,3]       [,4]        [,5]         [,6]
[1,] 0.005041357 0.020165428 0.03226468 0.02581175 0.010324699 0.0016519519
[2,] 0.016132342 0.064529370 0.10324699 0.08259759 0.033039037 0.0052862460
[3,] 0.019358811 0.077435244 0.12389639 0.09911711 0.039646845 0.0063434952
[4,] 0.010324699 0.041298797 0.06607807 0.05286246 0.021144984 0.0033831974
[5,] 0.002064940 0.008259759 0.01321561 0.01057249 0.004228997 0.0006766395


In [118]:
rounded_matrix <- round(m, digits = 4)

In [119]:
print(rounded_matrix)

       [,1]   [,2]   [,3]   [,4]   [,5]   [,6]
[1,] 0.0050 0.0202 0.0323 0.0258 0.0103 0.0017
[2,] 0.0161 0.0645 0.1032 0.0826 0.0330 0.0053
[3,] 0.0194 0.0774 0.1239 0.0991 0.0396 0.0063
[4,] 0.0103 0.0413 0.0661 0.0529 0.0211 0.0034
[5,] 0.0021 0.0083 0.0132 0.0106 0.0042 0.0007


In [120]:
n_rows_AB <- nrow(m)
n_cols_AB <- ncol(m)
mask_AB <- outer(1:n_rows_AB, 1:n_cols_AB, FUN = function(i, j) (((j-1) / 5) - ((i-1) / 4)))
print(mask_AB)
final_values_AB <- m[mask_AB]

print(final_values_AB)

      [,1]  [,2]  [,3]  [,4]  [,5] [,6]
[1,]  0.00  0.20  0.40  0.60  0.80 1.00
[2,] -0.25 -0.05  0.15  0.35  0.55 0.75
[3,] -0.50 -0.30 -0.10  0.10  0.30 0.50
[4,] -0.75 -0.55 -0.35 -0.15  0.05 0.25
[5,] -1.00 -0.80 -0.60 -0.40 -0.20 0.00


ERROR: Error in m[mask_AB]: only 0's may be mixed with negative subscripts


In [121]:
n_rows_AB <- nrow(m)
n_cols_AB <- ncol(m)
mask_AB <- outer(1:n_rows_AB, 1:n_cols_AB, FUN = function(i, j) (((j-1) / 5) - ((i-1) / 4))  >= obs_diff_A)
print(mask_AB)
final_values_AB <- m[mask_AB]

print(final_values_AB)

      [,1]  [,2]  [,3]  [,4]  [,5]  [,6]
[1,] FALSE FALSE  TRUE  TRUE  TRUE  TRUE
[2,] FALSE FALSE FALSE  TRUE  TRUE  TRUE
[3,] FALSE FALSE FALSE FALSE FALSE  TRUE
[4,] FALSE FALSE FALSE FALSE FALSE FALSE
[5,] FALSE FALSE FALSE FALSE FALSE FALSE
[1] 0.032264685 0.025811748 0.082597593 0.010324699 0.033039037 0.001651952
[7] 0.005286246 0.006343495


In [122]:
 sum(final_values_AB)

[1] 0.1973195

In [123]:
pobsC = 1/4

In [124]:
pestD = (13/6)/(20/6)
pestD

[1] 0.65

In [125]:
obs_diff_CD = pestD-pobsC
print(obs_diff_CD)

[1] 0.4


In [126]:
pCD = (1+13/6)/(4+20/6)
print(pCD)

[1] 0.4318182


In [127]:
pcombD = 1/6 + pCD*4/6
print(pcombD)

[1] 0.4545455


In [128]:
lC <- 0:4
print(lC)
lista_C <- numeric(length(lC))

for (i in lC) {
  lista_C[i + 1] <- dbinom(i, size = 4, prob = pCD)
}

print(lista_C)

[1] 0 1 2 3 4
[1] 0.10421958 0.31682753 0.36118338 0.18299958 0.03476992


In [129]:
lD <- 0:5

lista_D <- numeric(length(lD))

for (i in lD) {
  lista_D[i + 1] <- dbinom(i, size = 5, prob = pcombD)
}

print(lista_D)

[1] 0.04828284 0.20117851 0.33529751 0.27941460 0.11642275 0.01940379


In [130]:
m<- outer(lista_C, lista_D, FUN = "*")
print(m)

            [,1]        [,2]       [,3]        [,4]       [,5]         [,6]
[1,] 0.005032018 0.020966740 0.03494457 0.029120472 0.01213353 0.0020222550
[2,] 0.015297334 0.063738890 0.10623148 0.088526236 0.03688593 0.0061476553
[3,] 0.017438960 0.072662335 0.12110389 0.100919909 0.04204996 0.0070083270
[4,] 0.008835740 0.036815583 0.06135930 0.051132754 0.02130531 0.0035508857
[5,] 0.001678791 0.006994961 0.01165827 0.009715223 0.00404801 0.0006746683


In [131]:
rounded_matrix <- round(m, digits = 4)

In [132]:
print(rounded_matrix)

       [,1]   [,2]   [,3]   [,4]   [,5]   [,6]
[1,] 0.0050 0.0210 0.0349 0.0291 0.0121 0.0020
[2,] 0.0153 0.0637 0.1062 0.0885 0.0369 0.0061
[3,] 0.0174 0.0727 0.1211 0.1009 0.0420 0.0070
[4,] 0.0088 0.0368 0.0614 0.0511 0.0213 0.0036
[5,] 0.0017 0.0070 0.0117 0.0097 0.0040 0.0007


Create the mask matrix based on the condition p3 - p2 > x


In [133]:
mask <- difference_matrix > 0.4

print("Mask matrix (p3 - p2 > observed_diff_p23):")
print(mask)

[1] "Mask matrix (p3 - p2 > observed_diff_p23):"
       [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9] [,10] [,11] [,12]
 [1,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
 [2,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE
 [3,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE
 [4,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE
 [5,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE
 [6,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
 [7,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
 [8,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [9,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[10,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[11,] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[12,] FALSE FAL

Apply the mask to the result matrix and sum the filtered values


In [135]:
filtered_values_23 <- as.numeric(result_matrix_p23[mask])

print("Filtered values_23:")
print(filtered_values_23)

sum(filtered_values_23)

[1] "Filtered values_23:"
 [1] 0.000302 0.002489 0.000196 0.001612 0.006134 0.000097 0.000796 0.003027
 [9] 0.007038 0.011159 0.000036 0.000294 0.001120 0.002605 0.004130 0.004714
[17] 0.000010 0.000079 0.000302 0.000701 0.001112 0.001269 0.001073 0.000002
[25] 0.000015 0.000056 0.000130 0.000206 0.000235 0.000199 0.000126 0.000000
[33] 0.000002 0.000006 0.000015 0.000023 0.000027 0.000023 0.000014 0.000000
[41] 0.000000 0.000000 0.000001 0.000001 0.000001 0.000001 0.000001


[1] 0.051379